# English Subtitles Level Prediction

# Baseline-модель

---

**Входные данные**

Таблица с искусственно сгенерированными текстами, содержащими исключительно слова одного уровня.  
Структура таблицы в точности соответствует таблице с данными о фильмах.

---

**Цель**

Создать baseline-модель, не зависящую от оценок экспертов, в качестве нижнего порога качества для последующих моделей.

---

**Задачи:**  

- оценить качество модели по тем же метрикам, которые будут использованы для оценки последующих моделей;
- сделать контрольную проверку на тестовой выборке.

---

## Intro

**Some explanations**

Permanent data tables named like: **data**.  
Temporary data tables named like: **df**.  

The code of the cells are as independent as possible from each other in order to freely manipulate the cells.

Intermediate conclusions are highlighted as follows:

> Intermediate conclusion.

---

## Initial

### Imports

Почистить от лишних импортов !!!

In [41]:
import pandas as pd
import numpy as np

import os
import json
import warnings
import joblib
from datetime import date, time, datetime
from time import time

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, SplineTransformer, FunctionTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion

from nltk.corpus import stopwords

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import f1_score

import optuna
from optuna.distributions import FloatDistribution, IntDistribution, CategoricalDistribution

### Constants

In [42]:
PATH_LOCAL = 'datasets/'                               # local path to data
PATH_REMOTE = '/datasets/'                             # remote path to data

CR = '\n'                                              # new line

TARGET = 'Level'                                       # target name
SCORING = 'f1_weighted'                                # target metric
VALID_FRAC = 0.0                                       # delayed sampling fraction
N_CV = 5                                               # number of folds during cross-validation

N_TRIALS = 20                                          # max of tries when Optuna optimization run
TIMEOUT = 1000                                         # max time when Optuna optimization run
RANDOM_STATE = RANDOM_SEED = RS = 66                   # random_state

In [43]:
ESTIMATOR_LIST = [                                     # estimators list (comment/uncomment estimator to skip/use)
#                   'DummyClassifier',
                  'RidgeClassifier',
                  'SGDClassifier',
                  'LinearSVC',
                 ]

### Functions

In [44]:
def custom_read_csv(file_name, separator=','):
    """
    reading dataset of .csv format:
       first from local storage;
       if unsuccessful from remote storage.
    """

    path_local = f'{PATH_LOCAL}{file_name}'
    path_remote = f'{PATH_REMOTE}{file_name}'
    
    if os.path.exists(path_local):
        return pd.read_csv(path_local, sep=separator)

    elif os.path.exists(path_remote):
        return pd.read_csv(path_remote, sep=separator)

    else:
        print(f'File "{file_name}" not found at the specified path ')

In [45]:
def var_name(var):
    """
    var name determination
    """
    return [name for name in globals() if globals()[name] is var][0]

In [46]:
def plot_Optuna(study, plot_kind='plot_slice', model_name=''):
    '''
    Дополнительная настройка оригинальных графиков Optuna.
    Например, на графике `plot_slice` изначально цвет точек зависел от номера итерации.
    Теперь они все одинакового цвета и полупрозрачные, лучше видны скопления точек.
    
    study: обученный объект класса OptunaSearchCV
    plot_kind: тип графика Optuna
    model_name: название модели
    '''
    
    if plot_kind == 'plot_slice':
        fig = optuna.visualization.plot_slice(study)
        fig.update_traces(
                          marker_color='Darkgrey',
                          marker_size=3,
                          marker_opacity=0.2,
                          marker_line_width=1,
                          marker_line_color='Black',
                         )
    
    elif plot_kind == 'plot_param_importances':
        fig = optuna.visualization.plot_param_importances(study)
        
    elif plot_kind == 'plot_optimization_history':
        fig = optuna.visualization.plot_optimization_history(study)
        fig.update_traces(
                          marker_size=5,
                          marker_opacity=0.3,
                          marker_line_width=1,
                          marker_line_color='Black',
                         )

    fig.update_layout(
                      title_text=model_name,
                      title_x=0,
                      font_size=10,
                     )    
    fig.show()

In [47]:
def add_model_metrics(models, X_train, Y_train, X_valid, Y_valid, cv=N_CV, scoring_list=['f1'], verbose=True):
    '''
    Принимает:
        датафрейм со списком моделей и их характеристиками;
        два датасета (features and target) – обучающую и валидационную выборки;
        параметр cv для cross_val_score;
        список метрик
        
    Для каждой модели в датафрейме добавляет указанные метрики для обоих датасетов.
    '''

    def cv_score(model, X, Y, scoring, cv):
        invert_koeff = -1 if scoring.split('_')[0] == 'neg' else 1   # инвертирование метрик с приставкой "neg_"
        return invert_koeff * cross_val_score(model, X, Y, scoring=scoring, cv=cv, n_jobs=-1).mean()
    
    if verbose: print(f'Performing cross_val_score with cv={N_CV} and scoring_list={scoring_list}:')
    
    for scoring in scoring_list:
    
        # результаты моделей на обучающей выборке (усреднение на кроссвалидации)
        if verbose: print(f'{scoring}_train...', end=' ')
        models[f'{scoring}_train'] = models.model.apply(cv_score, args=(X_train, Y_train, scoring, cv))
        if verbose: print('done')

        # результаты моделей на тестовой выборке (усреднение на кроссвалидации)
        if verbose: print(f'{scoring}_test...', end=' ')
        models[f'{scoring}_test'] = models.model.apply(cv_score , args=(X_valid, Y_valid, scoring, cv))
        if verbose: print('done')
    
    # оптимальные гиперпараметры
    models['best_params'] = models.study.apply(lambda model: model.best_params)
    
    return models

In [48]:
def extract_final_features(pipeline_model):
    '''
    Принимает пайплайн.
    Возвращает список признаков, на которых обучается финальный estimator пайплайна.
    '''
    feature_list = []
    
    for feature in pipeline_model.steps[-2][1].get_feature_names_out():
        feature_list.append(feature.split('__')[1])

    return feature_list

### Settings

In [49]:
# text styles
class f:
    BOLD = "\033[1m"
    ITALIC = "\033[3m"
    END = "\033[0m"

In [50]:
# defaults for charts

# Matplotlib, Seaborn
PLOT_DPI = 150  # dpi for charts rendering 
sns.set_style('whitegrid', {'axes.facecolor': '0.98', 'grid.color': '0.9', 'axes.edgecolor': '1.0'})
plt.rc(
       'axes',
       labelweight='bold',
       titlesize=16,
       titlepad=10,
      )

# Plotly Graph_Objects
pio.templates['my_theme'] = go.layout.Template(
                                               layout_autosize=True,
                                               # width=900,
                                               layout_height=200,
                                               layout_legend_orientation="h",
                                               layout_margin=dict(t=40, b=40),         # (l=0, r=0, b=0, t=0, pad=0)
                                               layout_template='seaborn',
                                              )
pio.templates.default = 'my_theme'

# colors, color schemes
CMAP_SYMMETRIC = LinearSegmentedColormap.from_list('', ['steelblue', 'aliceblue', 'steelblue'])

In [51]:
# Pandas defaults
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 500
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.3f}'.format
pd.options.display.colheader_justify = 'left'

In [52]:
# оформление Optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)  # отключение вывода логов при работе optuna

In [53]:
# others
warnings.filterwarnings('ignore')

---

## Read and Check data

### Read data

In [54]:
data = custom_read_csv('EDA_artficial_subs.csv')

In [55]:
data.sample()

,Movie,Level,Subtitles
493,generated,C1,willingness reform lawn feat neglect formula judicial liver civic activation compensate memo inc...


In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Movie      500 non-null    object
 1   Level      500 non-null    object
 2   Subtitles  500 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


---

## Model

### Подготовка данных

#### Выделение признаков и целевой переменной

In [57]:
X = data.drop([TARGET, 'Movie'], axis=1)
Y = data[TARGET]

X.shape, Y.shape

((500, 1), (500,))

#### Разделение на обучающую и валидационную выборки

Валидационная выборка – часть, отрезанная от train, для локального тестирования модели.

*Примечание: если VALID_FRAC <= 0 для подбора гиперпараметров используется вся обучающая выборка (X, Y). Валидационная выборка в этом случае также равна (X, Y). Создание валидационной выборки в этом случае — лишь формальность (заглушка), и результаты валидации не следует интерпретировать никак.*

In [58]:
if VALID_FRAC > 0 :
    X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=VALID_FRAC, stratify=Y, random_state=RS)
else:
    X_train, X_valid, Y_train, Y_valid = X, X, Y, Y

X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape

((500, 1), (500,), (500, 1), (500,))

### Preprocessing

In [59]:
class TextSelector(BaseEstimator, TransformerMixin):
    '''
    Позволяет выбрать указанный признак для последующей обработки
    '''
    def __init__(self, field):
        self.field = field
    def fit(self, X, Y=None):
        return self
    def transform(self, X):
        return X[self.field]

#### Селекторы числовых и категориальных признаков

In [60]:
num_selector = make_column_selector(dtype_include=np.number)
cat_selector = make_column_selector(dtype_exclude=np.number)

#### Предбработка числовых признаков

In [61]:
num_preprocessor = make_pipeline(
                                 IterativeImputer(initial_strategy='mean', random_state=RS),
                                 StandardScaler(),
                                )

num_transformer = make_column_transformer(
                                          (num_preprocessor, num_selector),
                                          remainder='drop'
                                         )

#### Предбработка категориальных признаков

Категориальный признак только один – содержащий субтитры. По крайней мере, пока.

In [62]:
stopwords_english = list(set(stopwords.words('english')))

In [63]:
text_transformer = Pipeline([
                              ('TEXT_SELECT', TextSelector('Subtitles')),
                              ('TFIDF', TfidfVectorizer(
                                                        decode_error='ignore',
                                                        stop_words=stopwords_english,
                                                        token_pattern=r'(?u)\b[a-z]{3,}\b',      # токены из 3-х и более букв
                                                        smooth_idf=True)
                                                       ),
                             ])

#### Объединение предобработки числовых и категориальных признаков

In [64]:
features_union = FeatureUnion([
                               ('TEXT', text_transformer),
                               ('NUM', num_transformer),
                              ])

#### Pipelines' table

In [65]:
pipelines = [
             Pipeline([
                       ('FU', features_union),
                       ('DC', DummyClassifier())
                      ]),
    
             Pipeline([
                       ('FU', features_union),
                       ('RC', RidgeClassifier(random_state=RS))
                      ]),

             Pipeline([
                       ('FU', features_union),
                       ('SGDC', SGDClassifier(random_state=RS))
                      ]),

             Pipeline([
                       ('FU', features_union),
                       ('LSVC', LinearSVC(random_state=RS))
                      ]),
    
            ]

names = ['DummyClassifier', 'RidgeClassifier','SGDClassifier', 'LinearSVC']

short_names = ['DC', 'RC', 'SGDC', 'LSVC']

models = pd.DataFrame(
                      data={'name': names,
                            'short_name': short_names,
                            'model': pipelines,
                           },
                     )
models

,name,short_name,model
0,DummyClassifier,DC,"(FeatureUnion(transformer_list=[('TEXT',\n Pipeline(steps=[('TEXT..."
1,RidgeClassifier,RC,"(FeatureUnion(transformer_list=[('TEXT',\n Pipeline(steps=[('TEXT..."
2,SGDClassifier,SGDC,"(FeatureUnion(transformer_list=[('TEXT',\n Pipeline(steps=[('TEXT..."
3,LinearSVC,LSVC,"(FeatureUnion(transformer_list=[('TEXT',\n Pipeline(steps=[('TEXT..."


В таблице моделей необходимо оставить только те, что есть в списке ESTIMATOR_LIST (то есть удалить лишние).

In [66]:
for item in range(models.shape[0]):
    if models.loc[item,'name'] not in ESTIMATOR_LIST:
        models = models.drop(item, axis=0)
        
models = models.reset_index(drop=True)

models

,name,short_name,model
0,RidgeClassifier,RC,"(FeatureUnion(transformer_list=[('TEXT',\n Pipeline(steps=[('TEXT..."
1,SGDClassifier,SGDC,"(FeatureUnion(transformer_list=[('TEXT',\n Pipeline(steps=[('TEXT..."
2,LinearSVC,LSVC,"(FeatureUnion(transformer_list=[('TEXT',\n Pipeline(steps=[('TEXT..."


### Подбор гиперпараметров

#### Objective functions для Optuna

In [67]:
def objective_DC(trial):

    params = {
              'DC__strategy': trial.suggest_categorical('DC__strategy', ['most_frequent','prior','stratified','uniform']),
             }

    model.set_params(**params)
    cv_SKF = StratifiedKFold(n_splits=N_CV)
    
    return cross_val_score(model, X_train, Y_train, scoring=SCORING, cv=cv_SKF, n_jobs=-1).mean()

In [68]:
def objective_RC(trial):

    params = {
              'RC__alpha': trial.suggest_float('RC__alpha', 0.1, 10.0, log=True),
              'RC__class_weight': trial.suggest_categorical('RC__class_weight', [None,'balanced']),
              'RC__max_iter': trial.suggest_int('RC__max_iter', 100, 1000, log=True),
             }
    
    model.set_params(**params)
    cv_SKF = StratifiedKFold(n_splits=N_CV)
    
    return cross_val_score(model, X_train, Y_train, scoring=SCORING, cv=cv_SKF, n_jobs=-1).mean()

In [69]:
def objective_SGDC(trial):

    params = {
              'SGDC__loss': trial.suggest_categorical('SGDC__loss', ['hinge','log_loss']),
              'SGDC__alpha': trial.suggest_float('SGDC__alpha', 1e-5, 1e-1, log=True),
              'SGDC__class_weight': trial.suggest_categorical('SGDC__class_weight', [None,'balanced']),
              'SGDC__max_iter': trial.suggest_int('SGDC__max_iter', 100, 1000, log=True),
             }
    
    model.set_params(**params)
    cv_SKF = StratifiedKFold(n_splits=5)
    
    return cross_val_score(model, X_train, Y_train, scoring=SCORING, cv=cv_SKF, n_jobs=-1).mean()

In [70]:
def objective_LSVC(trial):

    params = {
              'LSVC__C': trial.suggest_float('LSVC__C', 0.1, 10),
              'LSVC__class_weight': trial.suggest_categorical('LSVC__class_weight', [None,'balanced']),
              'LSVC__max_iter': trial.suggest_int('LSVC__max_iter', 100, 1000, log=True),
              'LSVC__dual': trial.suggest_categorical('LSVC__dual', [True, False]),
             }
    
    model.set_params(**params)
    cv_SKF = StratifiedKFold(n_splits=5)
    
    return cross_val_score(model, X_train, Y_train, scoring=SCORING, cv=cv_SKF, n_jobs=-1).mean()

#### Optuna call

In [71]:
for item in range(models.shape[0]):
    
    print('—' * 60)
    print(f"{CR}{models.loc[item,'name']} hyperparams tuning...")
    
    model = models.loc[item,'model']

    # создание объекта optuna.study
    study = optuna.create_study(
                                study_name=models.loc[item,'name'],
                                direction="maximize",
                                sampler=optuna.samplers.TPESampler(seed=RS)
                               )

    # оптимизация (подбор гиперпараметров)
    study.optimize(eval(f"objective_{models.loc[item,'short_name']}"),
                   n_trials=N_TRIALS, timeout=TIMEOUT, show_progress_bar=True, n_jobs=-1)

    # извлечение и обучение лучшей модели – здесь можно сделать обучение на полном наборе данных (X,Y)
    model.set_params(**study.best_params).fit(X_train, Y_train)

    # сохранение результатов в таблице моделей
    models.loc[item,'model'] = model
    models.loc[item,'study'] = study
#     models.loc[item,'features'] = ', '.join(extract_final_features(model))   # пока недостаточно универсально
    models.loc[item,'score'] = study.best_value

    print(f'{CR}{f.BOLD}{study.study_name}{f.END}{CR}')
    print(f'Количество попыток: {len(study.trials)}')
    print(f'Лучший результат: {f.BOLD}{study.best_value:0.4f}{f.END}{CR}')
    print('Комбинация гиперпараметров:')
    print(json.dumps(study.best_params, indent=1, sort_keys=True), f'{CR}')

————————————————————————————————————————————————————————————

RidgeClassifier hyperparams tuning...


  0%|          | 0/20 [00:00<?, ?it/s]


RidgeClassifier

Количество попыток: 20
Лучший результат: 1.0000

Комбинация гиперпараметров:
{
 "RC__alpha": 0.6219138124526465,
 "RC__class_weight": "balanced",
 "RC__max_iter": 608
} 

————————————————————————————————————————————————————————————

SGDClassifier hyperparams tuning...


  0%|          | 0/20 [00:00<?, ?it/s]


SGDClassifier

Количество попыток: 20
Лучший результат: 1.0000

Комбинация гиперпараметров:
{
 "SGDC__alpha": 0.0008344781339401781,
 "SGDC__class_weight": null,
 "SGDC__loss": "hinge",
 "SGDC__max_iter": 215
} 

————————————————————————————————————————————————————————————

LinearSVC hyperparams tuning...


  0%|          | 0/20 [00:00<?, ?it/s]


LinearSVC

Количество попыток: 20
Лучший результат: 1.0000

Комбинация гиперпараметров:
{
 "LSVC__C": 9.37134624912163,
 "LSVC__class_weight": null,
 "LSVC__dual": false,
 "LSVC__max_iter": 184
} 



## Анализ моделей

### Чтение тестовых данных (реальных фильмов с субтитрами)

In [72]:
data_test = custom_read_csv('EDA_movies_subtitles.csv')

In [73]:
X_test = data_test.drop([TARGET, 'Movie'], axis=1)
Y_test = data_test[TARGET]

X_test.shape, Y_test.shape

((347, 1), (347,))

### Additional metrics

В дополнение к основной метрике, полученной во время подбора гиперпараметров, можно сравнить метрики, рассчитанные с помощью кроссвалидации. Можно изменить параметры кроссвалидации.  
Метрики рассчитываются как для обучающей, так и валидационной выборки.

In [74]:
# дополнительные метрики моделей
models = add_model_metrics(models, X_train, Y_train, X_test, Y_test, cv=N_CV,
                           scoring_list=['f1_weighted','f1_macro','f1_micro'])

models.drop(['short_name','model','study','best_params'], axis=1)

Performing cross_val_score with cv=5 and scoring_list=['f1_weighted', 'f1_macro', 'f1_micro']:
f1_weighted_train... done
f1_weighted_test... done
f1_macro_train... done
f1_macro_test... done
f1_micro_train... done
f1_micro_test... done


,name,score,f1_weighted_train,f1_weighted_test,f1_macro_train,f1_macro_test,f1_micro_train,f1_micro_test
0,RidgeClassifier,1.000,1.000,0.643,1.000,0.648,1.000,0.661
1,SGDClassifier,1.000,1.000,0.635,1.000,0.631,1.000,0.650
2,LinearSVC,1.000,1.000,0.642,1.000,0.637,1.000,0.656


> Метрики, полученные для тестовой выборки, можно использовать в качестве baseline.